In [5]:
from matplotlib import cm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import LinearLocator, FormatStrFormatter

def plotSol(u, n):
    u = np.reshape(u, (n, n))
    
    X = np.arange(0, 1, 1 / n)
    Y = np.arange(0, 1, 1 / n)
    X, Y = np.meshgrid(X, Y)

    fig = plt.figure()
    ax = fig.gca(projection='3d')
    
    surf = ax.plot_surface(X, Y, u, cmap=cm.hot)
    
    plt.show()

In [13]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
xc = [0.5, 0.5]

def g(x):
    if (x[0] > 1 or x[0] < 0 or x[1] > 1 or x[1] < 0) :
        return 0
    return 1 - 4*np.exp(-20*np.linalg.norm(np.subtract(x, xc))**2)

def plotg(dimension):
    temp = [[0 for x in range(dimension)] for y in range(dimension)] 
    for i in range(dimension):
        for j in range(dimension):
            temp[i][j] = g((i/dimension, j/dimension))
    plotSol(temp, dimension)
    
#i = y, j = x
def iStoC(i, j, dim):
        xcord = (j % dim) / dim
        ycord = (i // dim) / dim
        return (xcord, ycord)

#S[y][x]
def init_S(n, kappa):
    S = np.zeros((n*n, n*n))
    h = 1/n
    for i in range(n*n):
        gn = g(iStoC(i+1, i, n))
        ge = g(iStoC(i, i+1, n))
        gs = g(iStoC(i-1, i, n))
        gw = g(iStoC(i, i-1, n))
        if (i < n):
            if (i == 0):
                S[i][i] = -(ge+gn)/2-h*kappa
                S[i][i+1] = ge/2
                S[i][i+n] = gn/2
            elif(i == n - 1):
                S[i][i] = -(gw+gn)/2-h*kappa
                S[i][i-1] = gw/2
                S[i][i+n] = gn/2
            else:
                S[i][i] = -gn-(ge+gw)/2-h*kappa
                S[i][i-1] = gw/2
                S[i][i+1] = ge/2
                S[i][i+n] = gn
                
        if (i >= n*(n-1)):
            if (i % n == 0):
                S[i][i] = -(ge+gs)/2-h*kappa
                S[i][i+1] = ge/2
                S[i][i-n] = gs/2
            elif ((i + 1) % n == 0):
                S[i][i] = -(gw+gs)/2-h*kappa
                S[i][i-1] = gw/2
                S[i][i-n] = gs/2
            else:
                S[i][i] = -gs-(ge+gw)/2-h*kappa
                S[i][i-1] = gw/2
                S[i][i+1] = ge/2
                S[i][i-n] = gs
                
        if (n <= i < n*(n-1)):
            S[i][i-n] = gs
            S[i][i+n] = gn
            if (i % n == 0):
                S[i][i] = -ge-(gs+gn)/2-h*kappa
                S[i][i+1] = ge
            elif ((i + 1) % n == 0):
                S[i][i] = -gw -(gs+gn)/2-h*kappa
                S[i][i-1] = gw
            else:
                S[i][i] = -gn-ge-gs-gw
                S[i][i-1] = gw
                S[i][i+1] = ge
    return S

def init_M(dim):
    D = np.ones(dim)
    D[[0,-1]] = 0.5
    D = np.concatenate((0.5*D,np.tile(D,dim-2),0.5*D))
    M = np.diag(D)
    return (1/(dim*dim))*M

def init_u0(n):
    u = np.zeros((n*n, 1))
    for i in range(0, n*n):
        ycord = (i // n) / n
        xcord = (i % n) / n
        x = (xcord, ycord)
        if (np.linalg.norm(x) < 0.05):
            u[i] = 0.1
    return u

def backwardEuler(M, S, lam, dt, u0, epsilon, dimension, maxiter):
    Minv = np.linalg.inv(M)
    MinvS = Minv @ S
    ukn = ukn1 = u0
    sol = np.zeros((maxiter, dimension*dimension))
    for n in range(maxiter):
        print("iter", n)
        norm = np.inf
        while(norm > epsilon):
            uk1n1 = np.linalg.solve((np.eye(dimension*dimension) - dt*MinvS), (ukn + dt*lam*(ukn1 * (np.ones(ukn1.shape)-ukn1))))
            norm = np.linalg.norm(ukn1 - uk1n1)
            ukn1 = uk1n1
            print(norm)
        ukn = ukn1
        
        sol[n,:] = ukn[:,0]
    
    for i in range(maxiter):
        if (i % 100 == 0):
            plotSol(sol[i], dimension)

def forwardEuler(M, S, lam, dt, u0, dimension, maxiter):
    Minv = np.linalg.inv(M)
    MinvS = Minv @ S
    
    un = un1 = u0
    sol = np.zeros((maxiter, dimension*dimension))
    for n in range(maxiter):
        un1 = un + dt*MinvS.dot(un) + dt*lam*(un*(1-un))
        un = un1
        sol[n,:] = un[:,0]
    
    for i in range(maxiter):
        if (i % 100 == 0):
            plotSol(sol[i], dimension)
    
#print(init_S(10, 1))
# plt.spy(init_S(6,1))
#plt.matshow(init_S(100,1))
dimension = 20
#backwardEuler(init_M(dimension), init_S(dimension, 1), 10, 0.0001, init_u0(dimension), 1e-6, dimension, 30)
#forwardEuler(init_M(dimension), init_S(dimension, 1), 10, 0.0000001, init_u0(dimension), dimension, 300000)
#plotg(50)